In [ ]:
!pip install ezdxf
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
#connect to google drive
import os, sys
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import ezdxf
import shutil

def convertDXF(id):

  inpath = '/content/drive/Shareddrives/SmartCLT_DF2023/Scan2/GOOD_SCAN/'
  outputpath = '/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT1/'

  # Load the image
  image_path = inpath + str(id)+ '_solution.png'
  img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

  # Flip the image vertically
  flipped_img = cv2.flip(img, 0)


  # Blur image to enhance edge detection
  flipped_img = cv2.GaussianBlur(flipped_img, (5, 5), 0)

  # Apply Canny edge detection
  edges = cv2.Canny(flipped_img,50,100)

  # Find contours
  contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Create a new DXF document.
  doc = ezdxf.new('R2010')
  msp = doc.modelspace()


  # Set the drawing's units to millimeters
  doc.header["$INSUNITS"] = 4
  drawing_units = doc.header["$INSUNITS"]
  print(drawing_units)
  if drawing_units == 4:  # Drawing units are in millimeters
      pixel_to_mm = 1  # 1 pixel = 1 millimeter

  # Set layers
  doc = ezdxf.new(setup=True)  # setup required line types
  msp = doc.modelspace()
  doc.layers.add(name="Curve_Exist", color=63, linetype="Continuous")
  doc.layers.add(name="CuttingCurve_A", color=10, linetype="Continuous")
  doc.layers.add(name="CuttingCurve_B", color=30, linetype="Continuous")
  doc.layers.add(name="Image", color=191, linetype="Continuous")
  doc.layers.add(name="Solution", color=191, linetype="Continuous")


  move_distance = 303
  buffer = 0.025

  # Import background image
  fpath= inpath + str(id) + '_image.jpg'
  fpath1= inpath + str(id)+ '_solution.png'
  my_image_def = doc.add_image_def(filename=fpath, size_in_pixel=(3500/(1-2*buffer), 606))
  my_image_def1 = doc.add_image_def(filename=fpath1, size_in_pixel=(3500, 695))

  # Use shutil.copy() to duplicate the file to the new path
  shutil.copy(fpath, outputpath)
  shutil.copy(fpath1, outputpath)

  msp = doc.modelspace()

  msp.add_image(
      insert=(-move_distance, -3500/(1-2*buffer)*buffer),
      size_in_units=(3500/(1-2*buffer),606),
      image_def=my_image_def,
      rotation=0,
      dxfattribs={"layer": "Image"}
  )

  msp.add_image(
      insert=(-move_distance, 0),
      size_in_units=(3500,695),
      image_def=my_image_def1,
      rotation=0,
      dxfattribs={"layer": "Solution"}
  )


  # Convert contour points to polyline and reverse list
  contour_points = [contour.squeeze().tolist() for contour in contours]  # turn array tuple into a list
  polylst = []
  for i in range(len(contour_points)):
    contour_points2=[[contour_points[i][0][0]-move_distance,contour_points[i][0][1]]]
    ref = 0
    for points in contour_points[i]:
      points = [points[0]-move_distance,points[1]]
      #print(points)
      newpoint = points[1]
      if newpoint > ref:
        contour_points2.append(points)
        ref = newpoint
    print(i,contour_points2)


    polylst.append(contour_points2)
  polylst.reverse()
  print(polylst)

  # Assigning layers
  for i in range(len(polylst)):
    drawpolylst=polylst[i]
    if i==0:
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "Curve_Exist"})
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "CuttingCurve_A"})
    elif i==1:
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "CuttingCurve_B"})
    elif i==2:
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "Curve_Exist"})

  # Save the document.
  outputname = str(id) + '_DF_poly.dxf'
  outputfile = outputpath + outputname
  doc.saveas(outputfile)

In [ ]:
convertDXF('08')

4
0 [[130, 0], [130, 2], [129, 3], [129, 5], [128, 6], [128, 33], [127, 34], [127, 56], [126, 57], [126, 59], [125, 60], [125, 81], [124, 82], [124, 83], [123, 84], [123, 99], [122, 100], [122, 116], [121, 117], [121, 118], [120, 119], [120, 136], [119, 137], [119, 139], [118, 140], [118, 155], [117, 156], [117, 177], [115, 179], [115, 200], [114, 201], [114, 221], [113, 222], [113, 224], [112, 225], [112, 241], [111, 242], [111, 244], [110, 245], [110, 267], [109, 268], [109, 291], [108, 292], [108, 293], [107, 294], [107, 321], [106, 322], [106, 324], [105, 325], [105, 345], [104, 346], [104, 360], [103, 361], [103, 362], [102, 363], [102, 379], [101, 380], [101, 384], [100, 385], [100, 386], [101, 387], [100, 388], [100, 389], [101, 390], [101, 392], [100, 393], [100, 396], [99, 397], [99, 413], [98, 414], [98, 416], [97, 417], [97, 436], [96, 437], [96, 453], [95, 454], [95, 455], [94, 456], [94, 480], [93, 481], [93, 483], [92, 484], [92, 501], [91, 502], [91, 523], [89, 525], [89

# For Test

In [ ]:
      elif i==2:
    msp.add_spline(contour_points2,dxfattribs={"layer": "CuttingCurve_B"})



    if i == 3:
      msp.add_spline(contour_points2,dxfattribs={"layer": "Curve_Exist"})
      contour_points3=[]
      for j in range(len(contour_points2)):
        po=[contour_points2[j][0],contour_points2[j][1]]
        contour_points3.append(po)
      print(contour_points3)
      msp.add_spline(contour_points3,dxfattribs={"layer": "CuttingCurve_A"})
  elif i==2:
    msp.add_spline(contour_points2,dxfattribs={"layer": "CuttingCurve_B"})
  elif i==1:
    msp.add_spline(contour_points2,dxfattribs={"layer": "Curve_Exist"})